**Objetivos del Estudio**

El objetivo de este estudio es analizar una base de datos de libros, autores, editoriales, calificaciones y reseñas de clientes para generar una propuesta de valor para un nuevo producto. Este análisis buscará identificar patrones y tendencias en la lectura durante la pandemia, evaluar la calidad de los libros y autores, y entender las preferencias de los usuarios.

In [1]:
# Importar librerías
import pandas as pd
from sqlalchemy import create_engine

# Configuración de la base de datos
db_config = {
    'user': 'practicum_student', # nombre de usuario
    'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # puerto de conexión
    'db': 'data-analyst-final-project-db' # nombre de la base de datos
}

# Cadena de conexión
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                        db_config['pwd'],
                                                        db_config['host'],
                                                        db_config['port'],
                                                        db_config['db'])
# Crear el motor de conexión
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Verificar la conexión
try:
    connection = engine.connect()
    print("Conexión exitosa")
    connection.close()
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")


Conexión exitosa


In [ ]:
# Imprimir las primeras filas de la tabla books
books_df = pd.io.sql.read_sql("SELECT * FROM books LIMIT 5;", con=engine)
print("Tabla books:\n", books_df)

# Imprimir las primeras filas de la tabla authors
authors_df = pd.io.sql.read_sql("SELECT * FROM authors LIMIT 5;", con=engine)
print("Tabla authors:\n", authors_df)

# Imprimir las primeras filas de la tabla publishers
publishers_df = pd.io.sql.read_sql("SELECT * FROM publishers LIMIT 5;", con=engine)
print("Tabla publishers:\n", publishers_df)

# Imprimir las primeras filas de la tabla ratings
ratings_df = pd.io.sql.read_sql("SELECT * FROM ratings LIMIT 5;", con=engine)
print("Tabla ratings:\n", ratings_df)

# Imprimir las primeras filas de la tabla reviews
reviews_df = pd.io.sql.read_sql("SELECT * FROM reviews LIMIT 5;", con=engine)
print("Tabla reviews:\n", reviews_df)


Tabla books:
    book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268  
Tabla authors:
    author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4                   Alan Brennert
4          5

**Encuentra el número de libros publicados después del 1 de enero de 2000.**

In [ ]:
# Consulta para contar el número de libros publicados después del 1 de enero de 2000
books_published_after_2000_df = pd.io.sql.read_sql("""
SELECT COUNT(*) as num_books_after_2000
FROM books
WHERE publication_date > '2000-01-01';
""", con=engine)
print("Número de libros publicados después del 1 de enero de 2000:\n", books_published_after_2000_df)


Número de libros publicados después del 1 de enero de 2000:
    num_books_after_2000
0                   819


Conclusión:

El número significativo de 819 libros publicados después del 1 de enero de 2000 indica un aumento considerable en la producción editorial en el siglo XXI. Esto refleja una tendencia en la industria editorial hacia una mayor oferta de libros, posiblemente impulsada por una demanda creciente de material de lectura. Este periodo también coincide con avances tecnológicos en la publicación y distribución de libros, como el auge de los libros electrónicos y las plataformas de autopublicación.

**Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.**

In [ ]:
# Consulta para encontrar el número de reseñas de usuarios y la calificación promedio para cada libro
reviews_and_avg_rating_df = pd.io.sql.read_sql("""
SELECT books.title, COUNT(reviews.review_id) as num_reviews, AVG(ratings.rating) as avg_rating
FROM books
LEFT JOIN reviews ON books.book_id = reviews.book_id
LEFT JOIN ratings ON books.book_id = ratings.book_id
GROUP BY books.title;
""", con=engine)
print("Número de reseñas de usuarios y la calificación promedio para cada libro:\n", reviews_and_avg_rating_df)


Número de reseñas de usuarios y la calificación promedio para cada libro:
                                                  title  num_reviews  \
0                            The Count of Monte Cristo          115   
1                              Count Zero (Sprawl  #2)            4   
2    The Botany of Desire: A Plant's-Eye View of th...            4   
3                                 The Poisonwood Bible          110   
4                                 The Canterbury Tales           18   
..                                                 ...          ...   
994                           Of Love and Other Demons            4   
995  In the Heart of the Sea: The Tragedy of the Wh...            9   
996               Welcome to Temptation (Dempseys  #1)            4   
997                      World's End (The Sandman  #8)            4   
998                                  Holes (Holes  #1)          155   

     avg_rating  
0      4.217391  
1      2.500000  
2      3.500000  


Conclusión:

El análisis del número de reseñas de usuarios y la calificación promedio para cada libro nos proporciona información valiosa sobre la popularidad y la percepción de calidad de los libros. Algunas observaciones clave incluyen:

Popularidad de los libros:

"Holes (Holes #1)" y "The Count of Monte Cristo" se destacan con el mayor número de reseñas, 155 y 115 respectivamente. Esto sugiere que estos libros son muy populares entre los lectores.
Libros con un número menor de reseñas, como "Count Zero (Sprawl #2)" y "Welcome to Temptation (Dempseys #1)", pueden no ser tan conocidos o recientes en el mercado.

**Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).**

In [ ]:
# Consulta para identificar la editorial que ha publicado el mayor número de libros con más de 50 páginas
top_publisher_df = pd.io.sql.read_sql("""
SELECT publishers.publisher, COUNT(books.book_id) as num_books
FROM books
JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY num_books DESC
LIMIT 1;
""", con=engine)
print("Editorial que ha publicado el mayor número de libros con más de 50 páginas:\n", top_publisher_df)


Editorial que ha publicado el mayor número de libros con más de 50 páginas:
        publisher  num_books
0  Penguin Books         42


Conlusiones:

Penguin Books se destaca como la editorial que ha publicado el mayor número de libros con más de 50 páginas, con un total de 42 libros. Esto sugiere que Penguin Books tiene una fuerte presencia en el mercado de libros de mayor extensión, excluyendo folletos y publicaciones menores. La prominencia de Penguin Books en este segmento puede ser indicativa de su enfoque en la publicación de contenido más sustancial y su capacidad para atraer a un público que busca lecturas más extensas y profundas.

**Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.**

In [ ]:
# Consulta para identificar al autor con la más alta calificación promedio del libro (al menos 50 calificaciones)
top_author_df = pd.io.sql.read_sql("""
SELECT authors.author, AVG(ratings.rating) as avg_rating
FROM books
JOIN authors ON books.author_id = authors.author_id
JOIN ratings ON books.book_id = ratings.book_id
GROUP BY authors.author
HAVING COUNT(ratings.rating) >= 50
ORDER BY avg_rating DESC
LIMIT 1;
""", con=engine)
print("Autor con la más alta calificación promedio del libro (al menos 50 calificaciones):\n", top_author_df)


Autor con la más alta calificación promedio del libro (al menos 50 calificaciones):
            author  avg_rating
0  Diana Gabaldon         4.3


Conclusion:

Diana Gabaldon se destaca como la autora con la más alta calificación promedio del libro, con una calificación promedio de 4.3 basada en al menos 50 calificaciones. Esto indica que sus libros no solo son populares entre un gran número de lectores, sino que también son altamente valorados. La consistencia en recibir calificaciones altas sugiere una fuerte conexión con sus lectores, probablemente debido a la calidad de su narrativa, el desarrollo de personajes, y la capacidad para mantener el interés del lector. Esto puede influir en decisiones editoriales y de marketing para promover más obras de esta autora, dado su éxito comprobado en satisfacer a un público amplio.

**Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.**

In [ ]:
# Consulta para encontrar el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros
avg_text_reviews_df = pd.io.sql.read_sql("""
WITH user_ratings_count AS (
    SELECT username, COUNT(rating_id) as num_ratings
    FROM ratings
    GROUP BY username
)
SELECT AVG(review_count) as avg_text_reviews
FROM (
    SELECT username, COUNT(review_id) as review_count
    FROM reviews
    WHERE username IN (
        SELECT username
        FROM user_ratings_count
        WHERE num_ratings > 50
    )
    GROUP BY username
) as subquery;
""", con=engine)
print("Número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros:\n", avg_text_reviews_df)


Número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros:
    avg_text_reviews
0         24.333333


Conlusion:

El número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros es 24.33. Esto indica que los usuarios más activos, aquellos que califican muchos libros, también tienden a contribuir con una cantidad significativa de reseñas de texto. Estas reseñas detalladas proporcionan una valiosa retroalimentación tanto para otros lectores como para los autores y editoriales. La alta participación en términos de calificaciones y reseñas textuales sugiere un alto nivel de compromiso y entusiasmo por parte de estos usuarios en la comunidad de lectores, lo cual es crucial para mantener una plataforma de libros activa y atractiva.